## Notebook for spike sorting from .rhd data without using the Pipeline of mountainsort (mac channel has no pipeline)
Uses:
    - intan2kwik (https://github.com/zekearneodo/intan2kwik/blob/master/README.md)
    - mountainlab suite(https://github.com/flatironinstitute/mountainlab-js)
    - mountainsort https://github.com/flatironinstitute/mountainsort_examples/blob/master/README.md
    - mountainsort examples https://github.com/flatironinstitute/mountainsort_examples/blob/master/README.md

In [1]:
import os
import glob
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import subprocess
from importlib import reload

# mountainlab imports
from pipefinch.neural.convert import intan
from pipefinch.neural.sort.mountain import core as msc

import logging

# Setup the logger
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
ch.setFormatter(formatter)
logger.addHandler(ch)

logger.info('Logger set')

2018-10-30 13:00:15,099 - root - INFO - Logger set


In [2]:
os.environ['OMP_NUM_THREADS'] = '8'
os.environ['MKL_NUM_THREADS'] = '8'
os.environ['NUMEXPR_NUM_THREADS'] = '8'

### Session parameters and raw files

In [3]:
exp_base = os.path.abspath('/Users/zeke/experiment/')

file_sys = {'exp_base': os.path.abspath('/Users/zeke/experiment/')}

sess_par = {'bird': 'o3g3',
           'sess': 'awake_20180908_2925',
           'rec': 0}

file_sys_defaults = {'exp_base': os.path.abspath('/data/experiment')}


def file_names(sess_par, file_sys=None):
    f_s = unroll_file_sys(file_sys_defaults, file_sys)    
    
    f_n = {}
    f_n['raw'] = os.path.join(f_s['raw'], bird)

In [4]:
exp_base

'/Users/zeke/experiment'

In [5]:
raw_folder = os.path.join(exp_base, 'raw_data', sess_par['bird'], 'neural', sess_par['sess'])
logger.info('Raw folder {}'.format(raw_folder))
glob.glob(os.path.join(raw_folder, '*.rhd'))

2018-10-30 13:00:32,045 - root - INFO - Raw folder /Users/zeke/experiment/raw_data/o3g3/neural/awake_20180908_2925


['/Users/zeke/experiment/raw_data/o3g3/neural/awake_20180908_2925/raw_intan_180908_153446.rhd',
 '/Users/zeke/experiment/raw_data/o3g3/neural/awake_20180908_2925/raw_intan_180908_151446.rhd',
 '/Users/zeke/experiment/raw_data/o3g3/neural/awake_20180908_2925/raw_intan_180908_143446.rhd',
 '/Users/zeke/experiment/raw_data/o3g3/neural/awake_20180908_2925/raw_intan_180908_155446.rhd',
 '/Users/zeke/experiment/raw_data/o3g3/neural/awake_20180908_2925/raw_intan_180908_161446.rhd',
 '/Users/zeke/experiment/raw_data/o3g3/neural/awake_20180908_2925/raw_intan_180908_145446.rhd',
 '/Users/zeke/experiment/raw_data/o3g3/neural/awake_20180908_2925/raw_intan_180908_152446.rhd',
 '/Users/zeke/experiment/raw_data/o3g3/neural/awake_20180908_2925/raw_intan_180908_150446.rhd',
 '/Users/zeke/experiment/raw_data/o3g3/neural/awake_20180908_2925/raw_intan_180908_142446.rhd',
 '/Users/zeke/experiment/raw_data/o3g3/neural/awake_20180908_2925/raw_intan_180908_154446.rhd',
 '/Users/zeke/experiment/raw_data/o3g3/n

In [12]:
rhd_files = glob.glob(os.path.join(raw_folder, '*.rhd'))
rhd_files.sort()
pd_f = pd.DataFrame(rhd_files, columns=['f_path'])

pd_f['stamp'] = pd_f['f_path'].apply(lambda x: int(os.path.split(x)[-1].split('.')[0].split('_')[-1]))

# Select some of the files
sess_start = 0
sess_end = 235959
pd_session = pd_f.loc[(pd_f['stamp'] > sess_start) & (pd_f['stamp'] < sess_end), :]
pd_session.index.size

rhd_list = pd_session['f_path'].values.tolist()[:]

## Scripts for sorting

### prep the files with their nice formats, locations and names


In [13]:
rhd_list

['/Users/zeke/experiment/raw_data/o3g3/neural/awake_20180908_2925/raw_intan_180908_142446.rhd',
 '/Users/zeke/experiment/raw_data/o3g3/neural/awake_20180908_2925/raw_intan_180908_143446.rhd',
 '/Users/zeke/experiment/raw_data/o3g3/neural/awake_20180908_2925/raw_intan_180908_144446.rhd',
 '/Users/zeke/experiment/raw_data/o3g3/neural/awake_20180908_2925/raw_intan_180908_145446.rhd',
 '/Users/zeke/experiment/raw_data/o3g3/neural/awake_20180908_2925/raw_intan_180908_150446.rhd',
 '/Users/zeke/experiment/raw_data/o3g3/neural/awake_20180908_2925/raw_intan_180908_151446.rhd',
 '/Users/zeke/experiment/raw_data/o3g3/neural/awake_20180908_2925/raw_intan_180908_152446.rhd',
 '/Users/zeke/experiment/raw_data/o3g3/neural/awake_20180908_2925/raw_intan_180908_153446.rhd',
 '/Users/zeke/experiment/raw_data/o3g3/neural/awake_20180908_2925/raw_intan_180908_154446.rhd',
 '/Users/zeke/experiment/raw_data/o3g3/neural/awake_20180908_2925/raw_intan_180908_155446.rhd',
 '/Users/zeke/experiment/raw_data/o3g3/n

In [14]:
ss_folder = os.path.join('/Users/zeke/experiment/ss_data/', sess_par['bird'], sess_par['sess'])

file_paths, out_folder = msc.make_paths(ss_folder)
os.makedirs(ss_folder, exist_ok=True)
#todo:
# meta and pars from intan files et al
s_f = 20000
ds_params = {'samplerate': s_f}

with open(file_paths['params'], 'w') as fp:
    json.dump(ds_params, fp)
    logger.info('Created session par files {}'.format(file_paths['params']))
    
# create the mda file from the rhd_list
#intan_meta, samples_in = intan.intan_to_bin(rhd_list, file_paths['mda'], variant='mda')


2018-10-30 13:32:19,950 - root - INFO - Created session par files /Users/zeke/experiment/ss_data/o3g3/awake_20180908_2925/params.json


In [15]:
ss_folder

'/Users/zeke/experiment/ss_data/o3g3/awake_20180908_2925'

In [16]:
file_paths

{'mda': '/Users/zeke/experiment/ss_data/o3g3/awake_20180908_2925/raw.mda',
 'params': '/Users/zeke/experiment/ss_data/o3g3/awake_20180908_2925/params.json',
 'geom': '/Users/zeke/experiment/ss_data/o3g3/awake_20180908_2925/geom.csv',
 'filt': '/Users/zeke/experiment/ss_data/o3g3/awake_20180908_2925/sort_out/filt.mda.prv',
 'pre': '/Users/zeke/experiment/ss_data/o3g3/awake_20180908_2925/sort_out/pre.mda.prv',
 'firings': '/Users/zeke/experiment/ss_data/o3g3/awake_20180908_2925/sort_out/firings.mda',
 'firings_curated': '/Users/zeke/experiment/ss_data/o3g3/awake_20180908_2925/sort_out/firings_curated.mda',
 'cluster_metrics': '/Users/zeke/experiment/ss_data/o3g3/awake_20180908_2925/sort_out/cluster_metrics.json'}

In [17]:
msc.sort_dataset(file_paths=file_paths, adjacency_radius=-1,detect_threshold=3)

2018-10-30 13:32:27,648 - pipefinch.sort.mountain.comre - INFO - Bandpass filter


RUNNING: ml-run-process ephys.bandpass_filter --inputs timeseries:/Users/zeke/experiment/ss_data/o3g3/awake_20180908_2925/raw.mda --parameters freq_max:6000 freq_min:300 samplerate:20000 --outputs timeseries_out:/Users/zeke/experiment/ss_data/o3g3/awake_20180908_2925/sort_out/filt.mda.prv


2018-10-30 13:33:32,638 - pipefinch.sort.mountain.comre - INFO - Whitening


[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Preparing temporary outputs... ]
[ Initializing process ... ]
[ Running ... ] /Users/zeke/anaconda3/envs/mountain/bin/python3 /Users/zeke/anaconda3/envs/mountain/etc/mountainlab/packages/ml_ephys/preprocessing/preprocessing.py.mp ephys.bandpass_filter --timeseries=/Users/zeke/experiment/ss_data/o3g3/awake_20180908_2925/raw.mda --timeseries_out=/tmp/mountainlab-tmp/tempdir_ad4d59445c_HeEZuk/output_timeseries_out.mda --freq_max=6000 --freq_min=300 --samplerate=20000 --_tempdir=/tmp/mountainlab-tmp/tempdir_ad4d59445c_HeEZuk
Chunk size: 30000, Padding: 3000, Num chunks: 4971, Num processes: 12

Processed 435 of 4971 chunks...
Processed 3507 of 4971 chunks...

Processed 3929 of 4971 chunks...
Processed 1806 of 4971 chunks...
Processed 1367 of 4971 chunks...
Processed 2639 of 4971 chunks..

2018-10-30 13:34:17,992 - pipefinch.sort.mountain.comre - INFO - Sorting


...

RUNNING: ml-run-process ms4alg.sort --inputs timeseries:/Users/zeke/experiment/ss_data/o3g3/awake_20180908_2925/sort_out/pre.mda.prv --parameters adjacency_radius:-1 detect_sign:1 detect_threshold:3 --outputs firings_out:/Users/zeke/experiment/ss_data/o3g3/awake_20180908_2925/sort_out/firings.mda


2018-10-30 13:47:20,638 - pipefinch.sort.mountain.comre - INFO - Getting cluster metrics


[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
Locating /Users/zeke/experiment/ss_data/o3g3/awake_20180908_2925/sort_out/pre.mda.prv ...
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Preparing temporary outputs... ]
[ Initializing process ... ]
[ Running ... ] /Users/zeke/anaconda3/envs/mountain/bin/python3 /Users/zeke/anaconda3/envs/mountain/etc/mountainlab/packages/ml_ms4alg/ms4alg_spec.py.mp ms4alg.sort --timeseries=/tmp/mountainlab-tmp/output_9d4ae8e0a31bf35c7dbf656592268f7abf62b244_timeseries_out.mda --geom= --firings_out=/tmp/mountainlab-tmp/tempdir_72eaf91f58_sSEnn8/output_firings_out.mda --adjacency_radius=-1 --detect_sign=1 --detect_threshold=3 --_tempdir=/tmp/mountainlab-tmp/tempdir_72eaf91f58_sSEnn8
Using tempdir=/tmp/mountainlab-tmp/tempdir_72eaf91f58_sSEnn8
Preparing /tmp/mountainlab-tmp/tempdir_72eaf91f58_sSEnn8/timeseries.hdf5...
Preparing neighborhood sorters

2018-10-30 13:47:53,599 - pipefinch.sort.mountain.comre - INFO - Automatically curating


[ Getting processor spec... ]
[ Checking inputs and substituting prvs ... ]
Locating /tmp/mountainlab-tmp/mountainlab/tmp_short_term/output_cluster_metrics_out_f2f39d1789fafe656a3aca12bd8d7b00baf6382d.prv ...
Locating /tmp/mountainlab-tmp/mountainlab/tmp_short_term/output_metrics_out_4168e01b0d6b7e44cfa32a12ea5600754535d0f1.prv ...
[ Computing process signature ... ]
[ Checking outputs... ]
[ Checking process cache ... ]
[ Creating temporary directory ... ]
[ Preparing temporary outputs... ]
[ Initializing process ... ]
[ Running ... ] /Users/zeke/anaconda3/envs/mountain/etc/mountainlab/packages/ms3.mp ms3.combine_cluster_metrics --metrics_list=/tmp/mountainlab-tmp/output_8b9197ae9a409f7b1cccc52e22ef5f890ac26221_cluster_metrics_out --metrics_list=/tmp/mountainlab-tmp/output_8469be9dbd814e451445466e3a2241b02a0bf787_metrics_out --metrics_out=/tmp/mountainlab-tmp/tempdir_b84e48e3c5_Pctg7b/output_metrics_out.json --_tempdir=/tmp/mountainlab-tmp/tempdir_b84e48e3c5_Pctg7b
Elapsed time for pr

### Command to visualize:
(go to corresponding ss_folder)

qt-mountainview --raw raw.mda --filt sort_out/filt.mda.prv --pre sort_out/pre.mda.prv --samplerate 20000 --firings sort_out/firings.mda

## Attempt to visualize from this notbeook (failed)

In [10]:
cmd = 'env'
p = subprocess.Popen([cmd], stdout=subprocess.PIPE)
ans = p.communicate()
print(ans)

(b'MANPATH=/opt/local/share/man/:\nTERM_PROGRAM=Apple_Terminal\nSHELL=/bin/bash\nTERM=xterm-color\nTMPDIR=/var/folders/gk/7z94s6yj40n4ggn5dl62xhbc0000gn/T/\nCONDA_SHLVL=1\nApple_PubSub_Socket_Render=/private/tmp/com.apple.launchd.rxM5aew32h/Render\nCONDA_PROMPT_MODIFIER=(mountain) \nTERM_PROGRAM_VERSION=404\nTERM_SESSION_ID=E6D6F9C0-AA53-473E-B5D3-B4EE02E408C3\nUSER=zeke\nCONDA_EXE=/Users/zeke/anaconda3/bin/conda\nML_CONDA_PACKAGES_DIR=/Users/zeke/anaconda3/envs/mountain/etc/mountainlab/packages\nSSH_AUTH_SOCK=/private/tmp/com.apple.launchd.ZGXuue9qT4/Listeners\nML_CONDA_DIR=/Users/zeke/anaconda3/envs/mountain/etc/mountainlab\nPATH=/Users/zeke/anaconda3/envs/mountain/bin:/Users/zeke/google-cloud-sdk/bin:/opt/local/bin:/opt/local/sbin:/Library/Frameworks/Python.framework/Versions/3.7/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin\nCONDA_PREFIX=/Users/zeke/anaconda3/envs/mountain\nPWD=/Users/zeke/repos\nML_CONFIG_DIRECTORY=/Users/zeke/anaconda3/envs/mountain/etc/mountainlab\nLANG=en_US

In [13]:
show_args_dict = {'raw': file_paths['mda'], 
             'filt': file_paths['filt'],
             'pre': file_paths['pre'],
            'samplereate': ds_params['samplerate'],
            'firings': file_paths['firings'],
            'cluster_metrics': file_paths['cluster_metrics']}

show_args_dict = {'raw': 'raw.mda', 
             'filt': 'filt.mda.prv',
             'pre': 'pre.mda.prf',
            'samplereate': ds_params['samplerate'],
            'firings': 'sort_out/firings.mda'}

cmd_args = ['--{} {}'.format(k, v) for k, v in show_args_dict.items()]
cmd = 'qt-mountainview'

logger.info('View command is {}'.format([cmd, ' '.join(cmd_args)]))
#p = subprocess.Popen([cmd, ' '.join(cmd_args)], stdout=subprocess.PIPE)


2018-10-18 11:28:34,310 - root - INFO - View command is ['qt-mountainview', '--raw raw.mda --filt filt.mda.prv --pre pre.mda.prf --samplereate 20000 --firings sort_out/firings.mda']


In [14]:
print(' '.join(['qt-mountainview'] + cmd_args))

qt-mountainview --raw raw.mda --filt filt.mda.prv --pre pre.mda.prf --samplereate 20000 --firings sort_out/firings.mda


# other viewing examples

ev-templates output/templates.mda.prv
ev-timeseries dataset/raw.mda.prv --firings output/firings.mda.prv --samplerate=30000
qt-mountainview --raw dataset/raw.mda.prv --filt output/filt.mda.prv --pre output/pre.mda.prv --samplerate 30000 --firings output/firings.mda.prv
